In [1]:
import os
import json
import requests
import re
import numpy as np
import faiss
import pickle
import gzip
from bs4 import BeautifulSoup
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai
from openai import OpenAI # API v2.0
#from openai.embeddings_utils import get_embedding, get_embeddings # before v1.2

with open("openai_key.txt", "r") as f:
    openai_key = f.read().strip()
    openai_client = OpenAI(api_key=openai_key)

In [14]:
# 定义函数

def crawler_page_urls():
    """# 爬取所有2866个连接 """
    urls0 = [f"https://gwins.org/cn/milesguo/list_2_{i}.html" for i in range(1,73)]
    urls1 = []
    for url in tqdm(urls0):
        response = requests.get(url)
        if response.status_code == 200:    
            soup = BeautifulSoup(response.content, 'html.parser')
            html_doc = soup.get_text() 
            link_string = '\n'.join([str(link) for link in soup.find_all('a')])
            pattern = r"/cn/milesguo/[\w/]+\.html"
            matches = re.findall(pattern, link_string)
            urls2 = [f"https://gwins.org{x}" for x in matches]
            urls1 += urls2
            print(len(urls1))
        else:
            print(f"无法获取页面{url}，HTTP状态码：{response.status_code}")
    return urls1

def download_documents():
    """# 爬取所有2866个文章，并保存为文档""" 
    urls1 = crawler_page_urls() # 爬取所有2866个连接
    out_folder = "./txts"
    if not os.path.isdir(out_folder): 
        os.mkdir("./txts")
    for url in tqdm(urls1):
        pattern = r'\d+'
        id = re.search(pattern, url).group() # 获取网页编号
        response = requests.get(url)
        if response.status_code == 200:    
            soup = BeautifulSoup(response.content, 'html.parser')
        else:
            print(f"无法获取页面{url}，HTTP状态码：{response.status_code}")
            continue
        html_doc = soup.get_text()  # 获取网页中的纯文本内容
        html_doc = re.sub(r'\s+', ' ', html_doc) # 去掉多余空格
        
        file_path = os.path.join(out_folder, f"{id}.txt")
        with open(file_path, "w") as f:
            f.write(html_doc) # 保存

In [15]:
def extract_titles(input_dir="./txts/", out_file="titles.json"):
    """提取每个文档的标题，并按照编号整理到一个json文件中"""
    files = [os.path.join(input_dir, x) for x in os.listdir(input_dir)]
    dict1 = dict()
    for in_file in tqdm(files):
        id = os.path.basename(in_file).split(".")[0]
        with open(in_file, "r") as f:
            txt = f.read()
        pattern1 = r'^(.*?)首页'
        match = re.match(pattern1, txt)
        if match: 
            title = match.group(1)
            title = title.replace("\n", " ")
        else:
            title = "unknown title"
        dict1[id] = title
        #print(title)
    with open(out_file, "w") as f:
        json.dump(dict1, f)

def load_titles(file="title.json"):
    """读取标题文件"""
    with open(file, "r") as f:
        dict1 = json.load(f)
    return dict1
    
def load_data_to_paragraphs(file):
    """ 将长文档分解成1000字以内短文档. 
    因为openai sentence embedding ada 002 8000 input token, 最大2000汉字
    但是逼近2000后语义编码效果会下降
    """
    with open(file, "r") as f:
        data = f.read()
    pattern1 = r'^.*?内容梗概: '
    pattern2 = r' 友情链接：Gnews \| Gclubs \| Gfashion \| himalaya exchange \| gettr \| 法治基金 \| 新中国联邦辞典 \| $'
    data = re.sub(pattern1, "", data)
    data = re.sub(pattern2, "", data)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    txts = text_splitter.split_text(data)
    return txts

def sentence_embedding_batch(txts, id):
    """将list of text编码为sentence embedding 1536维"""
    l1 = []
    #embs = get_embeddings(txts, engine="text-embedding-ada-002") # old api
    response = openai_client.embeddings.create(input = txts, model="text-embedding-ada-002")
    response = json.loads(response.json())["data"]
    embs = [x["embedding"] for x in response]
    for i, txt in enumerate(txts):
        label = f"{id}-{i}"
        emb = embs[i]
        l1.append((label, txt, emb))
    return l1

def encoding_file(in_file, output_dir):
    """将文档.txt文件分割并编码为sentence embedding，压缩保存为同名.npz文件"""
    id = os.path.basename(in_file).split(".")[0]
    out_file = os.path.join(output_dir, id+".npz")
    txts = load_data_to_paragraphs(in_file)
    packs = sentence_embedding_batch(txts, id)
    serialized_data = pickle.dumps(packs)
    compressed_data = gzip.compress(serialized_data)
    with open(out_file, "wb") as file:
        file.write(compressed_data)

def encoding_files(input_dir = "./txts/", output_dir = "./emb"):
    """批量将文件夹下txt文件编码为同名 embedding文件，2866个文件需要 openai 3美元"""
    files = [os.path.join(input_dir, x) for x in os.listdir(input_dir)]
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    for i, in_file in enumerate(tqdm(files)):
        encoding_file(in_file, output_dir)

def decoding_file(file):
    with open(file, "rb") as f:
        compressed_data = f.read()
    decompressed_data = gzip.decompress(compressed_data)
    l1 = pickle.loads(decompressed_data)
    return l1

def build_faiss_index(embs):
    d = 1536
    nlist = 100
    index = faiss.IndexFlatIP(d) # 普通向量内积暴力检索索引
    #index = faiss.IndexIVFFlat(index, d, nlist) # 倒排索引
    index.train(embs)
    index.add(embs)
    return index

def build_veactor_search_index(folder="./emb"):
    """构建向量检索索引和字典，充当向量数据库功能"""
    files = [os.path.join(folder, x) for x in os.listdir(folder)]
    dict_emb = dict()
    i = 0
    embs = []
    for file in tqdm(files):
        l1 = decoding_file(file)
        for idx, txt, emb in l1:
            dict_emb[i] = {"idx": idx, "txt": txt, "emb": emb}
            embs.append(emb)
            i+=1
    embs = np.vstack(embs)
    embs /= np.linalg.norm(embs, ord=2, axis=-1, keepdims=True) + 1e-8 # L2归一化
    faiss_index = build_faiss_index(embs)
    return embs, dict_emb, faiss_index

def text_search(query, faiss_index, dict_emb, dict_title, k=3):
    """ 
    用query文本，访问openai sentence embedding ada 002，得到句向量
    每1000个token(250汉字) 0.0001美元
    在向量索引中检索语音最相近的文档，并找对对应标题。
    """
    if len(query)<10:
        query = f"这是一个关于{query}的句子"
    #emb_query = get_embedding(query, engine="text-embedding-ada-002")
    response = openai_client.embeddings.create(input=[query], model="text-embedding-ada-002")
    emb_query = json.loads(response.json())["data"][0]["embedding"]
    emb_query = np.array(emb_query).reshape((1, -1))
    D, I = faiss_index.search(emb_query, k)
    txts = []
    for i in I[0]:
        idx = dict_emb[i]["idx"]
        idx0 = idx.split("-")[0]
        txt = dict_emb[i]["txt"]
        title = dict_title[idx0]
        txts += [(title, txt)]
    return txts

In [16]:
#download_documents() # 爬虫
#encoding_files(input_dir = "./txts/", output_dir = "./emb") # 编码，保存到文件
#extract_titles(input_dir="./txts/", out_file="titles.json") # 提取标题，保存到文件
embs, dict_emb, faiss_index = build_veactor_search_index(folder="./emb") # 读取编码文件，构建向量索引
dict_title = load_titles(file="titles.json") # 读取标题文件

100%|██████████████████████████████████████████████████████████████████████████████| 2866/2866 [00:06<00:00, 465.35it/s]


In [97]:
txt_query = "李克强的结局会是怎样？"
txts_retrival = text_search(txt_query, faiss_index, dict_emb, dict_title, k=3) #请求10000次API成本1美金
txts_retrival

[(' 郭文贵2022年5月17日直播 20220517_1直播乱聊 ',
  '但是王岐山就选择李克强那边，应该是没想到吧。王岐山和曾全走到李克强那边去了，这是江和曾和这个共青团，绝对是水火不容啊是吧？李克强是胡锦涛的人啊，而且习也是恨这个李克强，但你能想到吗现在？王岐山、曾庆红通通地走到李克强那去 就连朱镕基，当年朱镕基当时当总理的时候，李克强只要一说话，只要一说话，朱镕基就骂他。就朱镕基欺负他，欺负到了要死的程度。现在朱镕基也投靠李克强了，你想想这是什么概念啊？兄弟姐妹们。 所以说这个问题，它已经不是政治的问题啦，它是一个生死的选择。大家想想，就是抓个稻草，但是稻草也是草，我觉得李克强连个稻草都不算。我觉得李克强连稻草都不算，现在共产党抓了一个连稻草都不算的李克强。 （看留言）李克强的接任可能性有多大？可能性有多大是吧？没什么可能性我觉得。谢谢Rachel！ 小王子：七哥那您说就现在整个20大前夕王岐山还有朱镕基、曾庆红他们整个投靠李克强，这个反映了党内现在政治斗争怎么样的一个情况？ 郭文贵先生：我觉得李克强是要绝对玩政治上绝不输习近平的，这我绝对有说话权力。但是李克强现在是一个没有选择下的一个这些人的选择。但是我觉得到最后的时候，可能估计都不会让李克强上去，如果有那天的话——习被灭的话，我会觉得李克强他腿没迈出去就给灭了就。 我觉得现在已经没有什么”上海帮“啦，现在严格讲曾家是吧，还有这个朱镕基家，所有这些人，谢谢！（工作人员取走水杯）所有这些人都是在保命，谁管你什么共产党不共产党。如果说现在说把共产党贬成猪党、狗党他也愿意，只要能保命，就是保命嘛。现在跟七哥当年说的一样保命是吧？保财、报仇，这都是所有人的心里想法，跟共产党已经没有半毛钱关系了。 但是人家会利用共产党是吧，利用这个组织和权力，就是抢的这个权力嘛。我觉得李克强是被所有人利用的一个人，但是最后我不相信他，有机会，很少的机会，很少。李克强强硬一下，李克强根本没有强硬，李克强做最后的垂死挣扎。 习绝对是有高人在帮他运作，就让他干。 郭文贵先生：好，咱今天试直播试到这啊，现在的感觉怎么样？我觉得很好，就那天我们直播的时候你看，现在已经是黑了吗？ 小王子：看一下有没有反光主——镜头？ 工作人员：稍稍有些。 郭文贵先生：如果那天，现在是几点？七点多吗？八点了都，那就太没问题啦！你干嘛要搞，就这样就行了嘛。

In [17]:
def RAG_chatbot(txt_query, txts_retrival):
    """
    k=3时token 4000左右，请求250次API成本1美金左右，成本和输入token数量，即和检索数量k成正比
    gpt-3.5-turbo-1106，是最新版3.5模型，最大token 16k
    需要试验其他开源LLM
    reference： https://openai.com/pricing#language-models
    """
    prompt = "\n\n".join([f"标题：{title}\n 正文：{txt}" for title, txt in txts_retrival])
    prompt = f"你需要先摘要并总结参考文本，再解答这个问题{txt_query} 以下是参考文本\n\n{txts_retrival}"
    
    response = openai_client.chat.completions.create(model="gpt-3.5-turbo-1106",
      messages=[{"role": "user", "content": prompt}])
    txt_response = json.loads(response.json())["choices"][0]["message"]["content"]
    return txt_response

txt_query = "郭文贵是谁？"
txts_retrival = text_search(txt_query, faiss_index, dict_emb, dict_title, k=3)
txt_response = RAG_chatbot(txt_query, txts_retrival)
print(txt_response)
print("以下是检索得到的原始参考文本")
for i, (title, txt) in enumerate(txts_retrival):
    print(f"{i+1}. {title}")
    print(txt)

郭文贵是一名中国人，被广泛认为是一个争议性的人物。他公开反对中国共产党，并通过社交媒体和其他渠道发表大量反对政府的言论。他声称自己因政治原因受到监禁，并强调了对中国政府的不满情绪。尽管郭文贵一直在公开场合中批判中共的体制和政策，但也造成了很多关于他本人的争议性传言，包括一些负面的诽闻和指控。
以下是检索得到的原始参考文本
1.  郭文贵2018年11月20日视频 20181120_4路德访谈班农、文贵先生：谈谈未来制裁中共王岐山盗国贼的一系列行为 
郭文贵先生：这个不知道，听说很多人被抓了，他们去查，抓很多人，说谁把这个水立方的1120给打出来了。我们好几个员工也被抓走了，怀疑我们。 路德：就为这事？ 郭文贵先生：我说跟我们什么关系啊？还有一个你注意到了吗？就在我们开始演讲的前20分钟，班农和郭文贵一下子网上释放了，就在我们讲完，就会议刚刚进行完…… 路德：还专门发了个推。 郭文贵先生：是吗？我没有时间（看），刚刚讲完马上又封掉了。什么概念？说明了党内的99的九千万党员和99.999的人民希望就那几个人完蛋。 Sarah：对。 郭文贵先生：中国今天所有的问题你们都要问问十八大以后的政府，我们要问问习近平，我们要问问王岐山，谁都不要问。 当然你孟建柱、傅政华、孙力军，你就是替他咬人的嘛。那么，班农先生和律师团队、你知道和我说什么？我特别、当时几次问我，（我）一句话没说。因为他们外国人不会跟你想的，跟你虚虚实实，他都是来真的，说：“郭先生，我们律师为什么要放弃你这个？放弃你这个？除了香港我和你有利益冲突，代表着可能与海航有关系，我不能代表你。更重要的事情，所有人都告诉我们说 ‘你是习近平的一条狗，’ running dog，你是有政党冲突的关键人物，我们不想裹入这个政治。在新闻发布会以后，我继续代表你，所有的subpoena我帮你发上去，但这个会之前，我不可以。” 什么意思？人家共产党的宣传就是造谣：郭文贵是习近平的一条狗，是为了咬王岐山的。 但我今天，我8次提到习近平，我说：“习近平你知不知道海航这个钱移来移去，谁是背后的股东？你为什么不调查？” 我不反习近平，我不等于说我今天承认你呀！我从来没有承认过他呀，我也对他抱有希望啊，现在我全是失望啊。 Sarah：对。 郭文贵先生：我的家人在监狱里，我的钱被罚款，简直疯狂了！香港的法官、律政司都被他们威胁！律政司的人，对不